# PeDistance Linear Regression

### Initial settings

In [1]:
import numpy as np
import torch
from torch import nn
import pandas as pd
from torch.utils.tensorboard import SummaryWriter
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
    
def get_device():
    if torch.cuda.is_available():
        device = torch.device('cuda:0')
    else:
        device = torch.device('cpu') # don't have GPU 
    return device

# convert a df to tensor to be used in pytorch
def df_to_tensor(df):
    device = get_device()
    return torch.tensor(df.to_numpy()).float().to(device)

### Import Dataset

In [2]:
fields = [
    'Distance', 
    'PedTopLeftX', 
    'PedTopLeftY', 
    'PedTopRightX', 
    'PedTopRightY',
    'PedBottomLeftX', 
    'PedBottomLeftY',
    'PedBottomRightX',
    'PedBottomRightY'
]

df = pd.read_csv (r'./Dataset.csv', usecols=fields)
print(df.shape)

(689, 9)


### Splitting dataset


In [3]:
train, test = train_test_split(df, test_size = 0.3)

In [4]:
Y_training = df_to_tensor(train.Distance)
Y_testing = df_to_tensor(test.Distance)

print(Y_training.shape)
print(Y_testing.shape)

torch.Size([482])
torch.Size([207])


#### Getting width  height from raw 2D screen points

In [5]:
X_training = df_to_tensor(train.apply(lambda el :  pd.Series([abs(el.PedTopRightX - el.PedTopLeftX), abs(el.PedBottomLeftY - el.PedTopLeftY)], index=['width', 'height']), axis=1))
X_testing = df_to_tensor(test.apply(lambda el :  pd.Series([abs(el.PedTopRightX - el.PedTopLeftX), abs(el.PedBottomLeftY - el.PedTopLeftY)], index=['width', 'height']), axis=1))

print(X_training.shape)
print(X_testing.shape)

torch.Size([482, 2])
torch.Size([207, 2])


### Linear Regression Module

In [32]:
class LinearRegressor(nn.Module):
    def __init__(self, in_size, out_size):
        super(LinearRegressor, self).__init__()
        self.linear = nn.Linear(in_size,out_size)
        
    def fit(self, X_training, Y_training, lr= 0.001, epochs=200000, momentum=0.0):
        #normalizzazione dei dati
        self.means = X_training.mean(0)
        self.stds = X_training.std(0)
        X_training_norm = self._data_normalization(X_training)
        
        self.criterion = nn.MSELoss()
        self.optimizer = torch.optim.SGD(reg.parameters(),lr=lr, momentum=momentum)
        self.writer = SummaryWriter('logs/linear_regressor_lr' + str(lr) + '_momentum' + str(momentum) + '_epochs' + str(epochs))
        
        reg.train()
        for e in range(epochs):            
            preds_train = self.forward(X_training_norm)
            l = self.criterion(preds_train.view(-1),Y_training)

            self.writer.add_scalar('loss/train', l.item(), global_step=e)

            l.backward()

            self.optimizer.step()    
            self.optimizer.zero_grad()
            
#             reg.eval()
#             with torch.set_grad_enabled(False):
#                 preds_test = reg(X_testing_norm)
#                 l = criterion(preds_test.view(-1),Y_testing)
#                 writer.add_scalar('loss/test', l.item(), global_step=e)    
        
    def predict(self, X):  
        X_norm = self._data_normalization(X)
        return self.linear(X_norm)
        
    def score(self, X, Y):
        Y_preds = self.predict(X)
        l = self.criterion(Y_preds.view(-1),Y)
        return l, Y_preds
        
    def _data_normalization(self, data):
        return (data-self.means)/self.stds        
    
    def forward(self,x):
        result = self.linear(x)
        return result    

### Modular Linear Regression

In [ ]:
reg = LinearRegressor(2,1).to(get_device())
reg.fit(X_training, Y_training)

In [42]:
loss_train, Y_train_preds = reg.score(X_training, Y_training)
print("Loss train: " + str(loss_train.item()))
for p,c in zip(Y_training, Y_train_preds):
      print([p.item(), c.item()])

Loss train: 3.1563880443573
[10.049580574035645, 11.794140815734863]
[16.54785919189453, 16.204740524291992]
[17.499027252197266, 16.90924072265625]
[18.33592987060547, 16.558340072631836]
[7.945519924163818, 8.18791675567627]
[11.2643404006958, 12.702441215515137]
[7.011178016662598, 6.434742450714111]
[18.586181640625, 16.106216430664062]
[18.623069763183594, 17.29659080505371]
[19.500904083251953, 17.351253509521484]
[11.736872673034668, 12.495941162109375]
[10.745719909667969, 13.211240768432617]
[8.82455825805664, 10.161091804504395]
[11.05966567993164, 13.42854118347168]
[5.771786212921143, 3.3035926818847656]
[12.423357009887695, 12.810416221618652]
[18.069414138793945, 16.37883949279785]
[10.843029975891113, 12.591766357421875]
[7.358304023742676, 8.193317413330078]
[18.111705780029297, 16.73649024963379]
[14.637882232666016, 15.645990371704102]
[10.110217094421387, 12.550603866577148]
[18.104711532592773, 16.615703582763672]
[9.278579711914062, 11.77389144897461]
[11.866935729

In [38]:
loss_testing, Y_testing_preds = reg.score(X_testing, Y_testing)
print("Loss testing: " + str(loss_testing.item()))
for p,c in zip(Y_testing, Y_testing_preds):
    print([p.item(), c.item()])

Loss testing: tensor(2.5612, device='cuda:0', grad_fn=<MseLossBackward>)
[18.31271743774414, 17.092790603637695]
[5.4902873039245605, 3.987180709838867]
[8.036239624023438, 9.70962905883789]
[18.8629207611084, 17.073226928710938]
[19.131723403930664, 17.118440628051758]
[9.94034481048584, 11.259004592895508]
[19.398069381713867, 17.24732780456543]
[8.571755409240723, 11.093667030334473]
[15.247969627380371, 16.096765518188477]
[7.882311820983887, 8.640042304992676]
[19.570880889892578, 16.88562774658203]
[13.546939849853516, 14.829465866088867]
[13.94119644165039, 14.880078315734863]
[13.348835945129395, 13.827353477478027]
[18.93489646911621, 16.409202575683594]
[15.902115821838379, 16.189889907836914]
[17.412813186645508, 16.700727462768555]
[8.42554759979248, 9.016592025756836]
[19.55724334716797, 17.07997703552246]
[17.90846824645996, 16.848514556884766]
[8.028239250183105, 9.72852897644043]
[17.778383255004883, 16.6568660736084]
[19.567148208618164, 16.844465255737305]
[14.1994571

### Imperative Linear Regression

In [24]:
lr = 0.0001
momentum = 0.99
epochs = 200000

writer = SummaryWriter('logs/linear_regressor_lr' + str(lr) + '_momentum' + str(momentum) + '_epochs' + str(epochs))

#normalizzazione dei dati
means = X_training.mean(0)
stds = X_training.std(0)

X_training_norm = (X_training-means)/stds
X_testing_norm = (X_testing-means)/stds

reg = LinearRegressor(2,1).to(get_device())
criterion = nn.MSELoss()

optimizer = torch.optim.SGD(reg.parameters(),lr=lr, momentum=momentum)

for e in range(epochs):
    reg.train()
    preds_train = reg(X_training_norm)
    l = criterion(preds_train.view(-1),Y_training)

    writer.add_scalar('loss/train', l.item(), global_step=e)
    
    l.backward()
    
    optimizer.step()    
    optimizer.zero_grad()
        
    reg.eval()
 
    with torch.set_grad_enabled(False):
        preds_test = reg(X_testing_norm)
        l = criterion(preds_test.view(-1),Y_testing)
        writer.add_scalar('loss/test', l.item(), global_step=e)